# Línea base por semanas

La idea del notebook es tener una primera línea base para las estaciones de SIMAJ, el código puede funcionar como base para replicarlo a las estaciones de Mi Macro

In [130]:
import os
import sys
import osmnx as ox
import geopandas as gpd
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib as mpl
import numpy as np
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)
    import aqiGDL
%matplotlib inline

Colores y estilo de matplotlib a usar

In [131]:
plt.style.use('https://github.com/dhaitz/matplotlib-stylesheets/raw/master/pitayasmoothie-dark.mplstyle')
colors = ['7A76C2', 'ff6e9c98', 'f62196', '18c0c4', 'f3907e', '66E9EC']

Carga de estaciones de monitoreo

In [132]:
gdf_est = aqiGDL.gdf_from_db('estaciones_gdl','Estaciones')
gdf_est = ox.project_gdf(gdf_est,to_crs='EPSG:32613')
gdf_est_simaj = aqiGDL.gdf_from_db('estaciones_simaj','estaciones_simaj')
gdf_est_simaj = ox.project_gdf(gdf_est_simaj,to_crs='EPSG:32613')

Carga de las mediciones usando un rango de fechas

In [133]:
est = 'ATM'
inicio = '2013/21/31'
fin = '2019/12/31'
query = f"SELECT * FROM data.simaj_data_day WHERE \"FECHA\" between \'{inicio}\' and \'{fin}\'"
df = aqiGDL.df_from_query(query)
df['FECHA'] = pd.to_datetime(df['FECHA']).dt.date
df.sort_values(by=['FECHA'], inplace=True)

Red base de Guadalajara

In [134]:
G = ox.graph_from_bbox(20.751857,20.523110,-103.201328,-103.468643)
edges = ox.graph_to_gdfs(G, nodes=False)
edges = ox.project_gdf(edges,to_crs=gdf_est_simaj.crs)

Crea dataframe para incluir promedios semanales

In [135]:
df_week = pd.DataFrame(columns=['S_ID','S_YEAR','PARAM','EST_SIMAJ','CONC','LONG','LAT'])
year = [2014,2015,2016,2017,2018,2019]

i=0

for y in year:
    
    for s in range(1,53):
        
        for p in df.PARAM.unique():
            
            for est in df.EST_SIMAJ.unique():
            
                df_week.loc[i]=['S'+str(s),'S'+str(s)+'-'+str(y), p,
                                est, np.nan, df.loc[df.EST_SIMAJ==est]['LONG'].iloc[0],
                                df.loc[df.EST_SIMAJ==est]['LAT'].iloc[0]]

                i+=1
                
df_week['PROM'] = np.nan
df_week['DESV_EST'] = np.nan


Calcula promedios por semana

In [136]:
interval = 7

for p in df.PARAM.unique():
        
    for est in df.EST_SIMAJ.unique():
                    
        df_analysis = df.loc[(df.PARAM==p)&(df.EST_SIMAJ==est)]

        divide = int(round((len(df_analysis)/7),0))

        s = 1

        for i in range(0, divide):

            mean_conc = df_analysis.iloc[i*interval:i*interval+interval]['CONC'].mean()

            std_conc = df_analysis.iloc[i*interval:i*interval+interval]['CONC'].std()

            day_year = i*interval+int((((i*interval+interval)-(i*interval))/2)-0.5)
            
            year_df = df_analysis['FECHA'].iloc[day_year].year
            
            df_week.loc[(df_week.S_ID=='S'+str(s)) & 
                      (df_week.S_YEAR=='S'+str(s)+'-'+str(year_df)) & 
                      (df_week.PARAM==p) &
                        (df_week.EST_SIMAJ==est),
                        'PROM'] = mean_conc

            df_week.loc[(df_week.S_ID=='S'+str(s)) & 
                      (df_week.S_YEAR=='S'+str(s)+'-'+str(year_df)) & 
                      (df_week.PARAM==p) &
                        (df_week.EST_SIMAJ==est),
                        'DESV_EST'] = std_conc

            s += 1

            if s > 52:

                s = 0
                

Filtra para un año en específico

In [137]:
df_week17 = df_week.iloc[2600:5200]
df_week17[(df_week17['PARAM']=='SO2')&(df_week17['EST_SIMAJ']=='AGU')]

,S_ID,S_YEAR,PARAM,EST_SIMAJ,CONC,LONG,LAT,PROM,DESV_EST
2639,S1,S1-2015,SO2,AGU,NaN,-103.416431,20.631268,0.002163,0.000732
2689,S2,S2-2015,SO2,AGU,NaN,-103.416431,20.631268,0.002222,0.000461
2739,S3,S3-2015,SO2,AGU,NaN,-103.416431,20.631268,0.001592,0.000413
2789,S4,S4-2015,SO2,AGU,NaN,-103.416431,20.631268,0.001519,0.000509
2839,S5,S5-2015,SO2,AGU,NaN,-103.416431,20.631268,0.002141,0.000799
2889,S6,S6-2015,SO2,AGU,NaN,-103.416431,20.631268,0.001621,0.000554
2939,S7,S7-2015,SO2,AGU,NaN,-103.416431,20.631268,0.001183,0.000656
2989,S8,S8-2015,SO2,AGU,NaN,-103.416431,20.631268,0.001372,0.001433
3039,S9,S9-2015,SO2,AGU,NaN,-103.416431,20.631268,0.001912,0.000910
3089,S10,S10-2015,SO2,AGU,NaN,-103.416431,20.631268,0.000527,0.000884


## Plot 📈

In [117]:
errors = []
for est in df_week17.EST_SIMAJ.unique().tolist():
    try:
        fig, axes = plt.subplots(2,3,figsize=(16,9), sharex=True)
        for param, ax, color in zip(df_week17.PARAM.unique().tolist(), axes.flatten()[1:], colors):
            df_temp = df_week17[(df_week17['EST_SIMAJ']==est) & (df_week17['PARAM']==param)]
            ax.plot(df_temp['S_YEAR'], df_temp['PROM'], label=param, color='#'+color)
            ax.set_title(param,fontsize=15)
            if param == 'SO2' or param == 'PM10':
                ax.set_xticklabels([])
            else:
                ax.tick_params(labelrotation=45)
        
        a00 = axes[0,0]
        shax = a00.get_shared_x_axes()
        shax.remove(a00)
        edges.plot(ax=axes[0][0], color='#e8e9eb',linewidth=0.1, zorder=-1)
        edges[(edges['highway']=='primary') | (edges['highway']=='secondary')].plot(ax=axes[0][0], color='#e8e9eb',linewidth=0.5, zorder=0)
        gdf_est_simaj.plot(ax=axes[0][0], color='k', alpha=0.85, zorder=1)
        gdf_est_simaj[gdf_est_simaj['codigo']==est].plot(ax=axes[0][0], color='#ba0d38', alpha=0.85, zorder=2, markersize=90)
        axes[0][0].axis('off')
        estacion = gdf_est_simaj[gdf_est_simaj['codigo']==est]['nombre'].values[0]
        fecha_1 = df['FECHA'].min().strftime('%Y-%m-%d') 
        fecha_2 = df['FECHA'].max().strftime('%Y-%m-%d') 
        fig.suptitle(f'{estacion}\n{fecha_1} -- {fecha_2}', fontsize=20)
        plt.savefig(f'../output/figures/estsimaj/{est}_{fecha_1}.png',dpi=300)
        plt.close()
    except Exception as e:
        errors.append(est)
        pass

In [121]:
errors

[]

In [140]:
errors = []
for est in df_week17.EST_SIMAJ.unique().tolist():
    try:
        fig, axes = plt.subplots(2,3,figsize=(16,9), sharex=True)
        for param, ax, color in zip(df_week17.PARAM.unique().tolist(), axes.flatten()[1:], colors):
            df_temp = df_week17[(df_week17['EST_SIMAJ']==est) & (df_week17['PARAM']==param)]
            ax.plot(df_temp['S_YEAR'], df_temp['PROM'], label=param, color='#'+color)

            #rellena el espacio entre desviaciones estandar
            ax.fill_between(range(len(df_temp)), 
                                   df_temp['DESV_EST']*-1+df_temp['PROM'], 
                                    df_temp['DESV_EST']*1+df_temp['PROM'], 
                                   facecolor='#9ddc9b', 
                                   alpha=0.25)
            
            ax.set_title(param,fontsize=15)
            '''if param == 'SO2' or param == 'PM10':
                ax.set_xticklabels([])
            else:
                ax.tick_params(labelrotation=45)'''
            
            ax.tick_params(
                            axis='x',          # changes apply to the x-axis
                            which='both',      # both major and minor ticks are affected
                            bottom=False,      # ticks along the bottom edge are off
                            top=False,         # ticks along the top edge are off
                            labelbottom=False) # labels along the bottom edge are off
                
        
        a00 = axes[0,0]
        shax = a00.get_shared_x_axes()
        shax.remove(a00)
        edges.plot(ax=axes[0][0], color='#e8e9eb',linewidth=0.1, zorder=-1)
        edges[(edges['highway']=='primary') | (edges['highway']=='secondary')].plot(ax=axes[0][0], color='#e8e9eb',linewidth=0.5, zorder=0)
        gdf_est_simaj.plot(ax=axes[0][0], color='k', alpha=0.85, zorder=1)
        gdf_est_simaj[gdf_est_simaj['codigo']==est].plot(ax=axes[0][0], color='#ba0d38', alpha=0.85, zorder=2, markersize=90)
        axes[0][0].axis('off')
        estacion = gdf_est_simaj[gdf_est_simaj['codigo']==est]['nombre'].values[0]
        #fecha_1 = df['FECHA'].min().strftime('%Y-%m-%d') 
        fecha_1 = '2015-01-01'
        #fecha_2 = df['FECHA'].max().strftime('%Y-%m-%d')
        fecha_2 = '2015-12-31'
        fig.suptitle(f'{estacion}\n{fecha_1} -- {fecha_2}', fontsize=20)
        plt.savefig(f'../output/figures/estsimaj/{est}_{fecha_1}.png',dpi=300)
        plt.close()
    except Exception as e:
        errors.append(est)
        pass